In [1]:
import torch

# Load Hidden states
import torch
tensor_fw_avg = torch.load('language_translate_fw_avg_output_sample0.pth')
tensor_bw_avg = torch.load('language_translate_bw_avg_output_sample0.pth')

tensor_fw_avg = torch.tensor(tensor_fw_avg)
tensor_bw_avg = torch.tensor(tensor_bw_avg)

## Load Data
import pandas as pd
csv_file_path = 'language_translate_fw_output_sample0.csv'
data = pd.read_csv(csv_file_path, header=0)
languages = data['3'].unique()
types = data[data['3'] == languages[0]]['1'].values
print(types)
print(data.columns)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 1 1 1 3 3 1 3 1 1 1 1 1 3 1 1 1 3 1 1 3 1 3 1 1 1 1 1 1 1 1 3 1 1 3 1 1 1
 1 1 1 3 1 1 3 3 1 1 1 3 1 3 3 3 1 3 3 1 1 1 3 3 1 3 3 1 1 1 3 3 1 3 1 1 3
 3 3 3 1 3 3 3 1 1 3 1 3 3 1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Index(['0', '1', '2', '3', '4', '5'], 

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size, latent_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, latent_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

def train_autoencoder(model, inputs, target, num_epochs = 20, lr_=0.001):
    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr_)

    losses = []  # List to store losses for plotting
    for epoch in range(num_epochs):
        running_loss = 0.0
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, target)  # Compute the reconstruction loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        # Store loss for plotting
        losses.append(loss.item())
        # print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(tensor_fw_avg)}")

    # print("Finished Training")
    return losses

def get_error(model, inputs, targets):
    criterion = nn.MSELoss(reduction='none')
    running_loss = 0.0
    outputs = model(inputs)
    loss = criterion(outputs, targets)  # Compute the reconstruction loss
    # print('loss:', loss.shape)
    loss_2_sum = torch.sum(loss, 1) / loss.shape[1]
    # print('loss_2_sum:', loss_2_sum.shape)
    outputs_sum = torch.sum(outputs, 1) #/ outputs.shape[1]
    # print('outputs_sum:', outputs_sum.shape)
    targets_sum = torch.sum(targets, 1) #/ targets.shape[1]
    # print('targets_sum:', targets_sum.shape)
    errors = loss_2_sum / (outputs_sum * targets_sum)
    # print('errors:', errors.shape)
    return torch.abs(errors)
    # return errors


In [3]:
import numpy as np
import pandas as pd

input_size = tensor_fw_avg.shape[1]  # Size of vectors
losses_all = {}
num_epochs = 300
latents = np.arange(8, 64, 8)
# latents = [64, 32]
matrices = np.empty((0, 5))

for lang in languages:
    for latent_size in latents:
        index = list(data[data['3'] == lang].index)
        fw = tensor_fw_avg[index]
        fw = torch.tensor(fw, requires_grad=True)
        bw = tensor_bw_avg[index]
        bw = torch.tensor(bw, requires_grad=True)

        model = Autoencoder(input_size, latent_size)
        losses = train_autoencoder(model, fw, bw, num_epochs=num_epochs)
        losses_all[lang + ' (hid=' + str(latent_size) + ')'] = losses

        errors = get_error(model, fw, bw).detach().numpy()
        ids = list(range(errors.shape[0]))
        langs = np.repeat(lang, errors.shape[0])
        latent_sizes = np.repeat(latent_size, errors.shape[0]) / fw.shape[1]
        matrix = np.vstack((ids, types, langs, errors, latent_sizes))
        matrices = np.concatenate((matrices, matrix.T), axis=0)
        
    print(lang)
# print(losses_all)

/scratch-local/bmehrparvar.6222928/ipykernel_2551418/4266787441.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  fw = torch.tensor(fw, requires_grad=True)
/scratch-local/bmehrparvar.6222928/ipykernel_2551418/4266787441.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bw = torch.tensor(bw, requires_grad=True)


German
Greek
Spanish
Persian
French
Hindi
Croatian
Italian
Korean
Dutch
Romanian
Russian
Turkish
Chinese


In [4]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def plot_error(losses, title):
    plt.clf()
    for key, value in losses.items():
        plt.plot(value, label=key)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title(title)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True, ncol=2)
    plt.savefig('loss.png', bbox_inches='tight')
    plt.show()

def plot_elbow(dataframe, title='Elbow chart', id_=0):
    plt.clf()
    # Get unique categories
    unique_categories = np.unique(dataframe[2].unique())
    num_categories = len(unique_categories)

    # Define a colormap with enough distinct colors
    cmap = plt.get_cmap('tab10', num_categories)
    norm = mcolors.Normalize(vmin=0, vmax=num_categories - 1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    scatter_plots = {}
    for id in dataframe[0].unique():
        if id == id_:
            for i, lang in enumerate(dataframe[2].unique()):
                if True: #lang == 'Persian':
                    df = dataframe[(dataframe[0] == id) & (dataframe[2] == lang)]

                    data = {'X': df[4].values,
                            'Y': df[3].values}
                    df = pd.DataFrame(data)

                    filter = df.sort_values(by='X')
                    # plt.scatter(filter[4].values, filter[3].values, color=cmap(i))
                    scatter_plots[lang], = plt.plot(filter['X'], filter['Y'],
                                                    linestyle='-', marker='o',
                                                    color=cmap(i), label=lang)

    plt.xlabel('Complexity (c)')
    plt.ylabel('error (e)')
    plt.title('%s item: %s' % (title, id_))
    plt.legend(handles=[scatter_plots[cat] for cat in unique_categories])
    plt.grid(True)
    plt.savefig('elbow_s%s.png' % id_)
    plt.show()

# plot_error(losses_all, 'Training Loss over Iterations')
df_matrices = pd.DataFrame(matrices)
df_matrices[0] = df_matrices[0].astype(int)
df_matrices[1] = df_matrices[1].astype(int)
df_matrices[2] = df_matrices[2].astype(str)
df_matrices[3] = df_matrices[3].astype(float)
df_matrices[4] = df_matrices[4].astype(float)

# plot_elbow(df_matrices, 'Elbow chart (training epochs=%s)' % num_epochs)
# print(df_matrices)

In [5]:
df_matrices.to_csv('elbow_avg_output_sample0.csv', index=False)

In [17]:
matrices_raw = np.empty((0, 2 + 2 * latents.shape[0] * len(languages)))
matrices_raw_best = np.empty((0, 2 + len(languages)))
mx = max(df_matrices[4].values)
print(mx)
for id in df_matrices[0].unique():
    filtered = df_matrices[df_matrices[0]==id]
    filtered_best = df_matrices[(df_matrices[0]==id) & (df_matrices[4]==mx)]
    row = np.hstack((id,filtered.iloc[0][1], filtered[3].T.values, filtered[4].T.values))
    matrices_raw = np.vstack((matrices_raw, row))
        
    row_best = np.hstack((id,filtered_best.iloc[0][1], filtered_best[3].T.values))
    matrices_raw_best = np.vstack((matrices_raw_best, row_best))

df_matrices_raw = pd.DataFrame(matrices_raw)
df_matrices_raw_best = pd.DataFrame(matrices_raw_best)
df_matrices_raw[0] = df_matrices_raw[0].astype(int)
df_matrices_raw[1] = df_matrices_raw[1].astype(int)
df_matrices_raw_best[0] = df_matrices_raw_best[0].astype(int)
df_matrices_raw_best[1] = df_matrices_raw_best[1].astype(int)
for i in range(len(df_matrices_raw.columns)):
    if i > 2:
        df_matrices_raw[i] = df_matrices_raw[i].astype(float)
for i in range(len(df_matrices_raw_best.columns)):
    if i > 2:
        df_matrices_raw_best[i] = df_matrices_raw_best[i].astype(float)
print(df_matrices_raw.shape)
print(df_matrices_raw_best.shape)
df_matrices_raw.to_csv('elbow_avg_output_sample0_raw.csv', index=False)
df_matrices_raw_best.to_csv('elbow_avg_output_sample0_raw_best.csv', index=False)

0.0546875
(474, 198)
(474, 16)


In [18]:
plot_error(losses_all, 'Training Loss over Iterations')

In [19]:
plot_elbow(df_matrices, 'Elbow chart (training epochs=%s)' % num_epochs, id_=0)
plot_elbow(df_matrices, 'Elbow chart (training epochs=%s)' % num_epochs, id_=237)